# Project Proposal

Group Number: 4

Group Members:Kaiji Lo, Hao Jiang, Jezarah Ebel, Yoson Hsu, Amar Gill

Data Set: Data Science Jobs Salaries

In [2]:
library(cowplot)
library(dplyr)
library(gridExtra)
library(tidyverse)
library(repr)
library(infer)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ stringr 1.4.0
✔ tidyr   1.2.0     ✔ forcats 0.5.1
✔ readr   2.1.2     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ gridExtra::combine() masks dplyr::combine()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::lag()         masks stats::lag()



## Introduction

During COVID-19 the work from home (WFH) model was adopted by many companies to keep employees safe from the virus; 32% of Canadian employees in 2021 worked from home compared to 4% in 2016 (Mehdi). In general, it has been proven that employee’s productivity is not affected if their personal work environment at home provides good privacy, noise, and allows developers to work with less fragmentation or interruptions (Bao). However, companies have in practice determined compensation, based on the location the employee is working from.

If switching from an in office to WFH model, how much can an employee in the Data Science industry expect their salary, on average (mean) and with what spread (standard deviation), to change by?

To explore this question, the dataset that we will be using, Data Science Jobs Salaries Dataset, includes salary data for employees in the data science industry such as the work year, experience level, employment type, the job title, salary, employee residence, the ratio of remote work, location of company, and size of company. To answer our questions, we will be looking at the salary in USD and the ratio of remote work (0% remote vs. 100% remote).


# Preliminary Results

## Cleaning and Wrangling of Data

In [3]:
download.file("https://raw.githubusercontent.com/Wills-Hao/201-Project/main/Data_Science_Jobs_Salaries.csv", destfile = "Data_Science_Jobs_Salaries.csv")

DsSalary <- read_csv("Data_Science_Jobs_Salaries.csv")
head(DsSalary)

Rows: 245 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (7): experience_level, employment_type, job_title, salary_currency, empl...
dbl (4): work_year, salary, salary_in_usd, remote_ratio

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
2021,EN,FT,Data Science Consultant,54000,EUR,64369,DE,50,DE,L
2020,SE,FT,Data Scientist,60000,EUR,68428,GR,100,US,L
2021,EX,FT,Head of Data Science,85000,USD,85000,RU,0,RU,M
2021,EX,FT,Head of Data,230000,USD,230000,RU,50,RU,L
2021,EN,FT,Machine Learning Engineer,125000,USD,125000,US,100,US,S
2021,SE,FT,Data Analytics Manager,120000,USD,120000,US,100,US,M


As the introduction mentioned, we are interested in the data-related workers working remotely or not (the remote ratio of 0 and 100). Since the remote ratio and the salary(USD) are the only columns we are interested in, we just select them for the cleaned data frame.

In [4]:
clean_DsSalary <- DsSalary %>%
                  filter(!is.na(salary_in_usd&remote_ratio),
                         remote_ratio %in% c(0, 100)) %>%
                  mutate(remote_ratio = as_factor(remote_ratio)) %>%
                  select(remote_ratio, salary_in_usd)
head(clean_DsSalary)

remote_ratio,salary_in_usd
<fct>,<dbl>
100,68428
0,85000
100,125000
100,120000
0,450000
100,144000


## Estimation of the Parameter

At first, we separately calculated the mean salaries of data-related workers who worked remotely (remote ratio > 80%) and the mean salaries of data-related workers who didn't work remotely (remote ratio < 20%). From the result below, we can see that the mean salary of remote workers has higher salaries. Then we find the difference between the two means is around 30145.229 USD, which can be seen as an estimation of the parameter.

In [5]:
remote_df <- clean_DsSalary %>%
                  filter(remote_ratio == 100) %>%
                  select(salary_in_usd)
 
remote_mean <- remote_df %>%
                summarise(mean_remote = mean(salary_in_usd))
remote_mean

no_remote_df <- clean_DsSalary %>%
                filter(remote_ratio == 0) %>%
                select(salary_in_usd)

no_remote_mean <- no_remote_df %>%
                 summarise(mean_office = mean(salary_in_usd))
no_remote_mean

mean_remote
<dbl>
115107.7


mean_office
<dbl>
84962.45


In [6]:
estimation_diff_mean <- pull(remote_mean) - pull(no_remote_mean)
estimation_diff_mean
options("scipen"=-100, "digits"=4)

[1] 30145.23

## Visualizing the Data

Firstly, we will plot the raw data distribution of two data frames(the remote working one and the no remote working one) with the vertical lines of their mean value. By comparing them, we can see that the mean salary of remote workers has a higher value of salary.

In [12]:
remote_distribution <- remote_df %>%
                          ggplot(aes(x=salary_in_usd)) +
                          geom_histogram(bins=80, color="darkblue", fill="lightblue") +
                          geom_vline(aes(xintercept = pull(remote_mean)), color="red", linetype="dashed", size = 1)+
                          ggtitle("Sample distribution of the salaries(USD) of data related jobs worked remotely (remote ratio > 80%)")+
                          labs(x="Salaries (in USD)",
                               y="People")+
                          theme(text = element_text(size=20)) 

no_remote_distribution <- no_remote_df %>%
                          ggplot(aes(x=salary_in_usd)) +
                          geom_histogram(bins=80, color="darkblue", fill="lightblue") +
                          geom_vline(aes(xintercept = pull(no_remote_mean)), color="red", linetype="dashed", size = 1)+
                          ggtitle("Sample distribution of the salaries(USD) of data related jobs worked in office (remote ratio < 20%)")+
                          labs(x="Salaries (in USD)",
                               y="People")+
                          theme(text = element_text(size=20))

options(repr.plot.width = 16, repr.plot.height = 14)

mean_plot_row <- plot_grid(remote_distribution+
                           scale_x_continuous(limits = c(0, 6.5e+05)),
                           no_remote_distribution+
                           scale_x_continuous(limits = c(0, 6.5e+05)),
                           nrow = 2)
mean_plot_row

Warning message:
“Removed 2 rows containing missing values (geom_bar).”


ERROR: Error in valid.charjust(just): invalid horizontal justification


In [ ]:

remote_density <- remote_df %>%
                          ggplot(aes(x=salary_in_usd)) +
                          geom_histogram(aes(y=..density..), bins=80, color="darkblue", fill="lightblue") +
                          geom_density(alpha=.2, fill="#FF6666")+
                          labs(x = "Salaries (in USD)",
                               y = "Density")+
                          ggtitle("Density plot of working remotely (remote ratio > 80%)")+
                          theme(text = element_text(size=20))
no_remote_density <- no_remote_df %>%
                          ggplot(aes(x=salary_in_usd)) +
                          geom_histogram(aes(y=..density..), bins=80, color="darkblue", fill="lightblue") +
                          geom_density(alpha=.2, fill="#FF6666")+
                          labs(x = "Salaries (in USD)",
                               y = "Density")+
                          ggtitle("Density plot of working in office (remote ratio < 20%)")+
                          theme(text = element_text(size=20))

options(repr.plot.width = 20, repr.plot.height = 8)

density_plot <- plot_grid(remote_density+
                           scale_x_continuous(limits = c(0, 6.5e+05)),
                           no_remote_density+
                           scale_x_continuous(limits = c(0, 6.5e+05)),
                           ncol = 2)
density_plot

From the density plots, we can see both of them are skewed to the right.

# Hopothesis Testing

For this section, we will use hypothesis testing to determine whether there is enough statistical evidence in favor of the statement that the mean salary for data scientists who work remotely is different from the mean salary for data scientists who does not work remotely. For this project, we set the 0.95 `confidence level`, which means a 0.05 significance level.

The null hypothesis ($H_0$): There is `no difference` between the mean salary for data scientists who work remotely and the mean salary for data scientists who does not work remotely.

The alternative hypothesis ($H_A$): There is `a difference` between the mean salary for data scientists who work remotely and the mean salary for data scientists who does not work remotely.


$\mu_1$: The population mean of salary for data scientists who `work remotely`

$\mu_2$: The population mean of salary for data scientists who does not `work remotely`

$H_0: \mu_1 - \mu_2 = 0$

$H_A: \mu_1 - \mu_2 \neq 0$

### Bootstrapping method

First, we will review the `observed test statistic` (the difference between the mean salary for data scientists who work remotely and the mean salary for data scientists who does not work remotely from the dataset) calculated in the preliminary results section. Then, we will use the `infer` package to obtain the bootstrapped sampling distribution to estimate the sampling distribution, so that we can use it to find out the `p-value` (the probability of getting a value more "extreme" than the observed test statistic). Also, we will construct a `confidence interval` based of the confidence level on the bootstrap distribution. Based on the p-value and the confidence interval, we can reject or not reject the null hypothesis $H_0$. 

In [8]:
# Review of the obeserved test statistic
cat("The value of the observed test statistic is", estimation_diff_mean)

The value of the observed test statistic is 3.015e+04

#### Find the null distribution and the p-value 

In [9]:
# Using the infer package, to obtain the bootstrapped sampling distribution of test statistic 
# by 1000 replications
set.seed(111)

sampling_dist_mean_salary <-  
   clean_DsSalary %>%
   specify(formula = salary_in_usd ~ remote_ratio) %>% 
   hypothesize(null = "independence") %>% 
   generate(reps = 1000, type = "permute") %>% 
   calculate(stat="diff in means", order = c(100, 0))

head(sampling_dist_mean_salary)

ERROR: Error: 1e+02 is not a level of the explanatory variable.


Since the alternative hypothesis is $\mu_1 - \mu_2 \neq 0$, we will find the p-value by the two sided direction (region shaded in red below). 

In [ ]:
# Plot the result of the hypothesis test with the vertical line of the test statistic's position

options(repr.plot.width = 8, repr.plot.height = 6)
result_plot_students <- 
   sampling_dist_mean_salary %>%
   visualize() + 
   shade_p_value(obs_stat = estimation_diff_mean, direction = "two-sided") +
   ggtitle("Null distribution")+
   labs(x = "Difference of Means (in USD)")+
   theme(text = element_text(size = 20))

result_plot_students

In [ ]:
# Obtaining the p-value

p_value_mean_salary <- 
    sampling_dist_mean_salary %>% 
    get_p_value(obs_stat = estimation_diff_mean, direction = "two-sided") %>%
    pull()

cat("The P-value is", p_value_mean_salary)

#### Find the bootstrap distribution and the 95% confidence interval

In [10]:
# Using the infer package, to obtain the bootstrap distribution of test statistic by 1000 replications
set.seed(222)

bootstrap_dist_diff_in_mean <-  
   clean_DsSalary %>%
   specify(formula = salary_in_usd ~ remote_ratio) %>% 
   generate(reps = 1000, type = "bootstrap") %>% 
   calculate(stat="diff in means", order = c(100, 0))

# Obtain the 95% confidence interval
diff_in_means_ci <- bootstrap_dist_diff_in_mean %>%
                    get_confidence_interval(level = 0.95, type = "percentile") 

diff_in_means_ci

ERROR: Error: 1e+02 is not a level of the explanatory variable.


In [11]:
# The Plot of the confidence interval in bootstrap distribution with a vertical line 
# of the value of H_0 (null value)

options(repr.plot.width = 8, repr.plot.height = 6)
diff_in_means_ci_plot <- bootstrap_dist_diff_in_mean %>%
                        visualize(bins = 10) +
                        shade_confidence_interval(endpoints = diff_in_means_ci) +
                        xlab("Difference of Means (in USD)")+
                        ggtitle("Bootstrap Distribution") +
                    geom_vline(xintercept = 0, color = "red", alpha=.3, lwd=2)+
                    annotate("text", x = -15000, 
                             y =300, label = "The Null value 0 is on \n the left side of the lower CI", 
                             color="red", size=4.8)+
                    theme(text = element_text(size = 20))
    
diff_in_means_ci_plot

ERROR: Error in check_for_nan(data, "visualize"): object 'bootstrap_dist_diff_in_mean' not found


#### Result of the bootstrap method:

Two methods are used to determine whether reject the null hypothesis (no difference), one is by comparing the p-value and the significance level, and another is by determining whether the confidence interval embraced the value of the null hypothesis which is 0 (null value). 

First of all, the `p-value` of 0.04 which is smaller than the `significant level` of 0.05 is statistically significant. It shows strong evidence against the null hypothesis because the probability of the null hypothesis being correct is less than 5%. Therefore, we reject the null hypothesis and accept the alternative hypothesis.

Secondly, the `null value` of the difference between two groups' mean salary is 0, which is not contained in the `confidence interval` (515.3215, 57918.65). It indicates that the null value 0 is not a reasonable value for the true difference in mean salaries between the two groups. Thus, we reject the null hypothesis and accept the alternative hypothesis.

In summary, there is a difference between the mean of salary for data scientists who work remotely and the mean of salary for data scientists who does not work remotely. 

## Expected outcomes and significance

In this project, we are expecting to find the salary difference between people who work within the office and people who work remotely, although this will vary with many other factors which we examine in this project. Due to COVID, a lot of companies will allow the employee to choose to work in the office or work remotely. However, our findings on the salary difference might impact people’s decisions regarding their working mode. Future analyses this could lead to include finding what other factors will affect a greater salary. Future analyses could even build off of our project by constructing a model of the salary difference for people who work hybrid vs people who work in the office. Of course, this cannot be done with our model as there we only filter the people with a 100% remote ratio and people who work with a 0% remote ratio.


## Reference

Bao, L., Li, T., Xia, X. et al. How does working from home affect developer productivity? — A case study of Baidu during the COVID-19 pandemic. Sci. China Inf. Sci. (2022). https://doi.org/10.1007/s11432-020-3278-4

Mehdi, T. Morissette, R. Working from home: Productivity and preference. Statistics Canada. (2021).
https://publications.gc.ca/collections/collection_2021/statcan/45-28/CS45-28-1-2021-12-eng.pdf
